In [1]:
import pandas as pd
import numpy as np
import os
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [2]:
installments_payments = pd.read_csv('installments_payments.csv')

In [3]:
#Encoding categorical variables
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
def encoding_cat_vars(df,le):
    le_count = 0
    one_hot_count = 0
    # Iterate through the columns
    for col in df:
        if df[col].dtype == 'object' or df[col].dtype =='bool' :
            # If 2 or fewer unique categories
            if len(list(df[col].unique())) <= 2:
                # Train on the training data
                le.fit(df[col])
                # Transform both training and testing data
                df[col] = le.transform(df[col])    
                # Keep track of how many columns were label encoded
                le_count += 1
            else:
                one_hot_count +=1
    print ("no of columns label encoded : "+str(le_count))
    #One Hot encoding
    df = pd.get_dummies(df)
    print ("no of columns one hot encoded encoded : "+str(one_hot_count))
    return df

In [4]:
## Missing values###########
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns
###############################

In [5]:
#Removing highly correlated field in TMP_APP_TRAIN & TEST
def cor_find(df,threshold):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return corr_matrix,upper,to_drop

In [6]:
#Check for missing values
missing_values = missing_values_table(installments_payments)
missing_values['column_name'] = missing_values.index
missing_values = missing_values.reset_index(drop = True)
missing_values

Your selected dataframe has 8 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values,column_name
0,2905,0.0,DAYS_ENTRY_PAYMENT
1,2905,0.0,AMT_PAYMENT


In [7]:
installments_payments[installments_payments['DAYS_ENTRY_PAYMENT'].isnull()].head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
3764207,1531600,103793,1.0,7,-668.0,NaN,49741.020,NaN
3764208,1947105,159974,1.0,24,-36.0,NaN,22849.515,NaN
3764209,1843773,167270,1.0,22,-20.0,NaN,48092.355,NaN
3764210,1691592,192536,1.0,5,-2561.0,NaN,7675.425,NaN
3764211,1531299,157088,0.0,11,-1847.0,NaN,67.500,NaN


In [13]:
installments_payments[installments_payments['SK_ID_PREV']==1531600].sort_values(by=['NUM_INSTALMENT_NUMBER'])

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
1230469,1531600,103793,1.0,1,-848.0,-866.0,49741.02,49741.02
3118659,1531600,103793,1.0,2,-818.0,-821.0,49741.02,49741.02
1367795,1531600,103793,1.0,3,-788.0,-785.0,49741.02,49741.02
802700,1531600,103793,1.0,4,-758.0,-752.0,49741.02,49738.50
3769072,1531600,103793,1.0,5,-728.0,NaN,49741.02,NaN
3799638,1531600,103793,1.0,6,-698.0,NaN,49741.02,NaN
3764207,1531600,103793,1.0,7,-668.0,NaN,49741.02,NaN
3794788,1531600,103793,1.0,8,-638.0,NaN,49741.02,NaN
3794772,1531600,103793,1.0,9,-608.0,NaN,49741.02,NaN
3774329,1531600,103793,1.0,10,-578.0,NaN,49741.02,NaN


In [8]:
previous_app = pd.read_csv('previous_application.csv')

In [9]:
previous_app[previous_app['SK_ID_PREV']==1531600]

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
799003,1531600,103793,Cash loans,49741.02,1012500.0,1100709.0,NaN,1012500.0,WEDNESDAY,10,Y,1,NaN,NaN,NaN,XNA,Approved,-878,Cash through the bank,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Country-wide,-1,XNA,36.0,middle,Cash X-Sell: middle,365243.0,-848.0,202.0,365243.0,365243.0,1.0


In [11]:
len(previous_app[(previous_app['NAME_CONTRACT_TYPE']=='Cash loans') & (previous_app['DAYS_FIRST_DUE'].isnull())])

458767

#### Find null records

In [31]:
installment_null = installments_payments[installments_payments['DAYS_ENTRY_PAYMENT'].isnull()]

In [36]:
installment_null_total_count = installment_null.groupby(['SK_ID_CURR'])['SK_ID_PREV'].count().reset_index()

In [39]:
installment_null_application = installment_null.groupby(['SK_ID_CURR'])['SK_ID_PREV'].nunique().reset_index()

In [42]:
installment_null_application = installment_null_application.rename(columns={'SK_ID_PREV':'INST_NUM_NULL_APPS'})
installment_null_total_count = installment_null_total_count.rename(columns={'SK_ID_PREV':'INST_NUM_NULL_INSTS'})

In [43]:
installment_null = pd.merge(installment_null_total_count,installment_null_application,on=['SK_ID_CURR'],how='inner')

#### New fields

In [45]:
installments_payments['DAYS_ENTRY_PAYMENT'] = installments_payments['DAYS_ENTRY_PAYMENT'].fillna(0)
installments_payments['AMT_PAYMENT'] = installments_payments['AMT_PAYMENT'].fillna(0)
installments_payments['DAYS_INSTALMENT'] = abs(installments_payments['DAYS_INSTALMENT'])
installments_payments['DAYS_ENTRY_PAYMENT'] = abs(installments_payments['DAYS_ENTRY_PAYMENT'])


In [46]:
installments_payments['DAYS_DIFF'] = installments_payments['DAYS_INSTALMENT'] - installments_payments['DAYS_ENTRY_PAYMENT']
installments_payments['AMT_DIFF'] = installments_payments['AMT_INSTALMENT'] - installments_payments['AMT_PAYMENT']
installments_payments['AMT_DAYS_DIFF']= installments_payments['AMT_DIFF']/(1+installments_payments['DAYS_DIFF'])

In [47]:
installments_payments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,DAYS_DIFF,AMT_DIFF,AMT_DAYS_DIFF
0,1054186,161674,1.0,6,1180.0,1187.0,6948.360,6948.360,-7.0,0.000,-0.0000
1,1330831,151639,0.0,34,2156.0,2156.0,1716.525,1716.525,0.0,0.000,0.0000
2,2085231,193053,2.0,1,63.0,63.0,25425.000,25425.000,0.0,0.000,0.0000
3,2452527,199697,1.0,3,2418.0,2426.0,24350.130,24350.130,-8.0,0.000,-0.0000
4,2714724,167756,1.0,2,1383.0,1366.0,2165.040,2160.585,17.0,4.455,0.2475


##### Find mean,max, min, sum & variance

In [9]:
installments_payments[installments_payments['DAYS_ENTRY_PAYMENT']>0].head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT


In [49]:
#Find average amount per client & latest
installments_payments_cnt = installments_payments[['SK_ID_CURR','SK_ID_PREV']]
installments_payments_cnt = installments_payments_cnt.groupby(['SK_ID_CURR']).count().reset_index()
installments_payments_cnt = installments_payments_cnt.rename(columns={'SK_ID_PREV':'CNT_INSTALLMENT_REC'})
installments_payments_cal = installments_payments[['SK_ID_CURR','NUM_INSTALMENT_VERSION','NUM_INSTALMENT_NUMBER'
                                                  ,'DAYS_INSTALMENT','DAYS_ENTRY_PAYMENT','AMT_INSTALMENT','AMT_PAYMENT',
                                                  'DAYS_DIFF','AMT_DIFF','AMT_DAYS_DIFF']]
installments_payments_mean = installments_payments_cal.groupby(['SK_ID_CURR']).max().reset_index()
installments_payments_max = installments_payments_cal.groupby(['SK_ID_CURR']).max().reset_index()
installments_payments_min = installments_payments_cal.groupby(['SK_ID_CURR']).min().reset_index()
installments_payments_sum = installments_payments_cal.groupby(['SK_ID_CURR']).sum().reset_index()
installments_payments_var = installments_payments_cal.groupby(['SK_ID_CURR']).var().reset_index()

In [51]:
installments_payments_mean = installments_payments_mean.rename(columns={'AMT_INSTALMENT':'MEAN_AMT_INSTALMENT',
                                                                        'DAYS_ENTRY_PAYMENT':'MEAN_DAYS_ENTRY_PAYMENT'
                                                                       ,'DAYS_INSTALMENT':'MEAN_DAYS_INSTALMENT'
                                                                       ,'AMT_PAYMENT':'MEAN_AMT_PAYMENT',
                                                                       'NUM_INSTALMENT_NUMBER':'MEAN_NUM_INSTALMENT_NUMBER',
                                                                       'NUM_INSTALMENT_VERSION':'MEAN_NUM_INSTALMENT_VERSION',
                                                                       'DAYS_DIFF':'MEAN_DAYS_DIFF',
                                                                       'AMT_DIFF':'MEAN_AMT_DIFF',
                                                                       'AMT_DAYS_DIFF':'MEAN_AMT_DAYS_DIFF'})


##### Rename column names 

In [53]:
installments_payments_max = installments_payments_max.rename(columns={'AMT_INSTALMENT':'MAX_AMT_INSTALMENT',
                                                                        'DAYS_ENTRY_PAYMENT':'MAX_DAYS_ENTRY_PAYMENT'
                                                                       ,'DAYS_INSTALMENT':'MAX_DAYS_INSTALMENT'
                                                                       ,'AMT_PAYMENT':'MAX_AMT_PAYMENT',
                                                                       'NUM_INSTALMENT_NUMBER':'MAX_NUM_INSTALMENT_NUMBER',
                                                                       'NUM_INSTALMENT_VERSION':'MAX_NUM_INSTALMENT_VERSION',
                                                                       'DAYS_DIFF':'MAX_DAYS_DIFF',
                                                                       'AMT_DIFF':'MAX_AMT_DIFF',
                                                                       'AMT_DAYS_DIFF':'MAX_AMT_DAYS_DIFF'})

installments_payments_min = installments_payments_min.rename(columns={'AMT_INSTALMENT':'MIN_AMT_INSTALMENT',
                                                                        'DAYS_ENTRY_PAYMENT':'MIN_DAYS_ENTRY_PAYMENT'
                                                                       ,'DAYS_INSTALMENT':'MIN_DAYS_INSTALMENT'
                                                                       ,'AMT_PAYMENT':'MIN_AMT_PAYMENT',
                                                                       'NUM_INSTALMENT_NUMBER':'MIN_NUM_INSTALMENT_NUMBER',
                                                                       'NUM_INSTALMENT_VERSION':'MIN_NUM_INSTALMENT_VERSION',
                                                                       'DAYS_DIFF':'MIN_DAYS_DIFF',
                                                                       'AMT_DIFF':'MIN_AMT_DIFF',
                                                                       'AMT_DAYS_DIFF':'MIN_AMT_DAYS_DIFF'})



In [54]:
installments_payments_sum = installments_payments_sum.rename(columns={'AMT_INSTALMENT':'SUM_AMT_INSTALMENT',
                                                                        'DAYS_ENTRY_PAYMENT':'SUM_DAYS_ENTRY_PAYMENT'
                                                                       ,'DAYS_INSTALMENT':'SUM_DAYS_INSTALMENT'
                                                                       ,'AMT_PAYMENT':'SUMAMT_PAYMENT',
                                                                       'NUM_INSTALMENT_NUMBER':'SUM_NUM_INSTALMENT_NUMBER',
                                                                       'NUM_INSTALMENT_VERSION':'SUM_NUM_INSTALMENT_VERSION',
                                                                       'DAYS_DIFF':'SUM_DAYS_DIFF',
                                                                       'AMT_DIFF':'SUM_AMT_DIFF',
                                                                       'AMT_DAYS_DIFF':'SUM_AMT_DAYS_DIFF'})

installments_payments_var = installments_payments_var.rename(columns={'AMT_INSTALMENT':'VAR_AMT_INSTALMENT',
                                                                        'DAYS_ENTRY_PAYMENT':'VAR_DAYS_ENTRY_PAYMENT'
                                                                       ,'DAYS_INSTALMENT':'VAR_DAYS_INSTALMENT'
                                                                       ,'AMT_PAYMENT':'VAR_AMT_PAYMENT',
                                                                       'NUM_INSTALMENT_NUMBER':'VAR_NUM_INSTALMENT_NUMBER',
                                                                       'NUM_INSTALMENT_VERSION':'VAR_NUM_INSTALMENT_VERSION',
                                                                       'DAYS_DIFF':'VAR_DAYS_DIFF',
                                                                       'AMT_DIFF':'VAR_AMT_DIFF',
                                                                       'AMT_DAYS_DIFF':'VAR_AMT_DAYS_DIFF'})


##### Join data sets. Result will be one record per CURR ID

In [63]:
installment_total_perclient = pd.merge(installments_payments_mean,installments_payments_var,on=['SK_ID_CURR'],how='inner')
installment_total_perclient = pd.merge(installment_total_perclient,installments_payments_sum,on=['SK_ID_CURR'],how='inner')
installment_total_perclient = pd.merge(installment_total_perclient,installments_payments_min,on=['SK_ID_CURR'],how='inner')
installment_total_perclient = pd.merge(installment_total_perclient,installments_payments_max,on=['SK_ID_CURR'],how='inner')

In [64]:
installment_total_perclient = pd.merge(installment_total_perclient,installment_null,on=['SK_ID_CURR'],how='left')

In [67]:
installment_total_perclient['INST_NUM_NULL_INSTS'] = installment_total_perclient['INST_NUM_NULL_INSTS'].fillna(0)
installment_total_perclient['INST_NUM_NULL_APPS'] = installment_total_perclient['INST_NUM_NULL_APPS'].fillna(0)

#### Check for missing values. Find SK_ID_CURR which has missing values. Check the number of records in application test data. Since there are only 120 records in application test. We are droping records with na values in next line

In [68]:
#Check for missing values
missing_values = missing_values_table(installment_total_perclient)
missing_values['column_name'] = missing_values.index
missing_values = missing_values.reset_index(drop = True)
missing_values

Your selected dataframe has 48 columns.
There are 13 columns that have missing values.


,Missing Values,% of Total Values,column_name
0,15354,4.5,VAR_AMT_DAYS_DIFF
1,972,0.3,VAR_NUM_INSTALMENT_VERSION
2,972,0.3,VAR_NUM_INSTALMENT_NUMBER
3,972,0.3,VAR_DAYS_INSTALMENT
4,972,0.3,VAR_DAYS_ENTRY_PAYMENT
5,972,0.3,VAR_AMT_INSTALMENT
6,972,0.3,VAR_AMT_PAYMENT
7,972,0.3,VAR_DAYS_DIFF
8,972,0.3,VAR_AMT_DIFF
9,208,0.1,SUM_AMT_DAYS_DIFF


In [69]:
installment_total_perclient = installment_total_perclient.drop(columns=['VAR_AMT_DAYS_DIFF'])

In [73]:
#Check for missing values
missing_values = missing_values_table(installment_total_perclient)
missing_values['column_name'] = missing_values.index
missing_values = missing_values.reset_index(drop = True)
missing_values

Your selected dataframe has 49 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values,column_name


In [71]:
installment_total_perclient = installment_total_perclient.fillna(-1)

In [72]:
installment_total_perclient['SUM_DPD'] = installment_total_perclient['SUM_DAYS_ENTRY_PAYMENT'] - installment_total_perclient['SUM_DAYS_INSTALMENT']
installment_total_perclient['SUM_DBD'] = installment_total_perclient['SUM_DAYS_INSTALMENT'] - installment_total_perclient['SUM_DAYS_ENTRY_PAYMENT']
installment_total_perclient['SUM_DPD'] = installment_total_perclient['SUM_DPD'].apply(lambda x: x if x > 0 else 0)
installment_total_perclient['SUM_DBD'] = installment_total_perclient['SUM_DBD'].apply(lambda x: x if x > 0 else 0)

In [74]:
installment_total_perclient.shape

(339587, 49)

In [75]:
installment_total_perclient.to_csv('tmp_installment.csv',index=False)